In [1]:
import pandas as pd 
import numpy as np
import sys

In [2]:
fact_df = pd.read_csv('dimensions/fact_table_m.csv')
date_dimension = pd.read_csv('dimensions/date_dimension.csv')
mobility_dimension = pd.read_csv('dimensions/mobility_data_dimension_dated.csv')
phu_dimension = pd.read_csv('dimensions/PHU_location_dimension.csv')

In [3]:
date_dimension.dtypes

date_key                  int64
date                     object
date_id                   int64
day_name                 object
day_name_abbrev          object
day_of_month              int64
day_of_week               int64
day_of_year               int64
holiday_name             object
is_holiday                 bool
is_weekday                 bool
is_weekend                 bool
month_abbrev             object
month_end_flag             bool
month_name               object
month_number              int64
quarter                   int64
quarter_name             object
quarter_short_name       object
season                   object
week_begin_date          object
week_begin_date_id        int64
week_num_in_month         int64
week_num_in_year          int64
year                      int64
year_and_month           object
year_and_month_abbrev    object
year_and_quarter         object
dtype: object

In [4]:
# Convert dates to date objects
date_dimension['date'] = pd.to_datetime(date_dimension['date'])
date_dimension['week_begin_date'] = pd.to_datetime(date_dimension['week_begin_date'])

In [5]:
mobility_dimension['date'] = pd.to_datetime(mobility_dimension['date'])

In [6]:
mobility_dimension.tail()

,mobility_key,province,subregion,date,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential
739,739,Ontario,Toronto Division,2021-02-02,-57.0,-24.0,-29.0,-70.0,-57.0,23.0
740,740,Ontario,Toronto Division,2021-02-03,-56.0,-24.0,-17.0,-70.0,-56.0,23.0
741,741,Ontario,Toronto Division,2021-02-04,-54.0,-17.0,-2.0,-69.0,-55.0,23.0
742,742,Ontario,Toronto Division,2021-02-05,-58.0,-27.0,-42.0,-70.0,-55.0,25.0
743,743,Ontario,Toronto Division,2021-02-06,-55.0,-19.0,-22.0,-63.0,-23.0,14.0


In [7]:
mobility_dimension.subregion.unique()

array(['Ottawa Division', 'Regional Municipality of Durham',
       'Regional Municipality of Halton', 'Regional Municipality of Peel',
       'Regional Municipality of York', 'Toronto Division'], dtype=object)

In [8]:
fact_df.head()

,Onset_date_ID,Reported_date_ID,Test_date_ID,Specimen_date_ID,Patient_ID,PHU_Location_ID,Mobility_ID,Special_Measures_ID,Weather_ID,resolved,unresolved,fatal
0,20200311,20201205,20201205,20200929,1,2253,-1,5,-1,yes,no,no
1,20200323,20201212,20201212,20201211,2,2270,-1,3,-1,yes,no,no
2,20200429,20201006,20201006,20200821,3,2270,-1,14,-1,yes,no,no
3,20201002,20201006,20201006,20201002,6,2230,-1,14,-1,yes,no,no
4,20200926,20201006,20201006,20200930,7,2270,-1,14,-1,yes,no,no


In [9]:
phu_dimension.head()

,PHU_location_key,PHU_id,PHU_name,Address,City,Postal_code,Province,URL,Latitude,Longitude
0,0,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,Ontario,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
1,1,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,Ontario,www.peelregion.ca/health/,43.647471,-79.708893
2,2,2270,York Region Public Health Services,17250 Yonge Street,Newmarket,L3Y 6Z1,Ontario,www.york.ca/wps/portal/yorkhome/health/,44.048023,-79.480239
3,3,2236,Halton Region Health Department,1151 Bronte Road,Oakville,L6M 3Ll,Ontario,www.halton.ca/For-Residents/Public-Health/,43.413997,-79.744796
4,4,2230,Durham Region Health Department,605 Rossland Road East,Whitby,L1N 0B2,Ontario,www.durham.ca/en/health-and-wellness/health-an...,43.898605,-78.940341


In [10]:
phu_dimension['PHU_name'].unique()

array(['Toronto Public Health', 'Peel Public Health',
       'York Region Public Health Services',
       'Halton Region Health Department',
       'Durham Region Health Department', 'Ottawa Public Health'],
      dtype=object)

In [11]:
phu_dimension.head()

,PHU_location_key,PHU_id,PHU_name,Address,City,Postal_code,Province,URL,Latitude,Longitude
0,0,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,Ontario,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
1,1,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,Ontario,www.peelregion.ca/health/,43.647471,-79.708893
2,2,2270,York Region Public Health Services,17250 Yonge Street,Newmarket,L3Y 6Z1,Ontario,www.york.ca/wps/portal/yorkhome/health/,44.048023,-79.480239
3,3,2236,Halton Region Health Department,1151 Bronte Road,Oakville,L6M 3Ll,Ontario,www.halton.ca/For-Residents/Public-Health/,43.413997,-79.744796
4,4,2230,Durham Region Health Department,605 Rossland Road East,Whitby,L1N 0B2,Ontario,www.durham.ca/en/health-and-wellness/health-an...,43.898605,-78.940341


In [12]:
phu_subregion_map = {
    'Ottawa Public Health' : 'Ottawa Division',
    'Durham Region Health Department' : 'Regional Municipality of Durham',
    'Halton Region Health Department' : 'Regional Municipality of Halton',
    'Peel Public Health' : 'Regional Municipality of Peel',
    'York Region Public Health Services' : 'Regional Municipality of York',
    'Toronto Public Health' : 'Toronto Division'
}

In [13]:
print(mobility_dimension.columns)

Index(['mobility_key', 'province', 'subregion', 'date',
       'retail_and_recreation', 'grocery_and_pharmacy', 'parks',
       'transit_stations', 'workplaces', 'residential'],
      dtype='object')


In [14]:
fact_table_mob_mapped = pd.DataFrame(columns=fact_df.columns)

In [15]:
len(fact_df)

148053

In [16]:
y = 112352
for idx, row in fact_df.iterrows():
    sys.stdout.write("\r{0} out of {1} generated...".format(idx,y))
    fact_table_row = []
    fact_table_row.append(row['Onset_date_ID'])
    fact_table_row.append(row['Reported_date_ID'])
    fact_table_row.append(row['Test_date_ID'])
    fact_table_row.append(row['Specimen_date_ID'])
    fact_table_row.append(row['Patient_ID'])
    fact_table_row.append(row['PHU_Location_ID'])
    
    # Grab the PHU name 
    phu_name = phu_dimension[phu_dimension['PHU_id']==row['PHU_Location_ID']]['PHU_name'].values[0]
    reported_date = date_dimension[date_dimension['date_id']==row['Reported_date_ID']]['date'].values[0]
    
    mobility_key = mobility_dimension[(mobility_dimension['date'] == reported_date) &
            (phu_subregion_map[phu_name] == mobility_dimension["subregion"])]['mobility_key'].values[0]
    
    fact_table_row.append(mobility_key)
    fact_table_row.append(row['Special_Measures_ID'])
    fact_table_row.append(row['Weather_ID'])
    fact_table_row.append(row['resolved'])
    fact_table_row.append(row['unresolved'])
    fact_table_row.append(row['fatal'])
    
    fact_table_mob_mapped.loc[len(fact_table_mob_mapped)] = fact_table_row
    

148052 out of 112352 generated...

In [17]:
fact_table_mob_mapped.to_csv('dimensions/fact_table_mob.csv', index=False)